In [9]:
from pynvml import smi

In [10]:
nvsmi = smi.nvidia_smi.getInstance()

In [11]:
nvsmi

In [12]:
result = nvsmi.DeviceQuery('memory.free, memory.total')

In [13]:
type(result)


dict

In [15]:
nvsmi.DeviceQuery([smi.NVSMI_UTILIZATION_GPU, smi.NVSMI_UTILIZATION_MEM] )

{'gpu': [{'utilization': {'gpu_util': 0, 'memory_util': 0, 'unit': '%'}},
  {'utilization': {'gpu_util': 0, 'memory_util': 0, 'unit': '%'}}]}

In [ ]:
def report_gpu_metric(value, type, instance_id, zone, project_id):
    series = monitoring_v3.types.TimeSeries()
    series.metric.type = 'custom.googleapis.com/{type}'.format(type=type)
    series.resource.type = 'gce_instance'
    series.resource.labels['instance_id'] = instance_id
    series.resource.labels['zone'] = zone
    series.resource.labels['project_id'] = project_id
    point = series.points.add()
    point.value.int64_value = value
    now = time.time()
    point.interval.end_time.seconds = int(now)
    point.interval.end_time.nanos = int(
        (now - point.interval.end_time.seconds) * 10**9)
    client.create_time_series(project_name, [series])

In [16]:
from google.cloud import monitoring_v3
from google.cloud.monitoring_v3.query import Query
import datetime

In [17]:
client = monitoring_v3.MetricServiceClient()
project_name = f'projects/jk-mlops-dev'
filter = 'metric.type = starts_with("custom.googleapis.com/opencensus")'


In [32]:
for descriptor in client.list_metric_descriptors(name=project_name, filter_=filter):
    print(f'Descriptor: {descriptor.name}')
    print(descriptor)

Descriptor: projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gpu_memory_utilization_distribution
name: "projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gpu_memory_utilization_distribution"
labels {
  key: "opencensus_task"
  description: "Opencensus task identifier"
}
metric_kind: CUMULATIVE
value_type: DISTRIBUTION
unit: "%"
description: "The distribution of gpu memory utilization"
display_name: "OpenCensus/gpu_memory_utilization_distribution"
type: "custom.googleapis.com/opencensus/gpu_memory_utilization_distribution"

Descriptor: projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gpu_utilization_distribution
name: "projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gpu_utilization_distribution"
labels {
  key: "opencensus_task"
  description: "Opencensus task identifier"
}
metric_kind: CUMULATIVE
value_type: DISTRIBUTION
unit: "%"
description: "The distribution of gpu utilization"
display_na

In [30]:
for descriptor in client.list_metric_descriptors(name=project_name, filter_=filter):
    print(f'Deleting: {descriptor.name}')
    client.delete_metric_descriptor(name=descriptor.name)

Deleting: projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gpu_memory_utilization_distribution
Deleting: projects/jk-mlops-dev/metricDescriptors/custom.googleapis.com/opencensus/gpu_utilization_distribution


In [37]:
project_id = 'jk-mlops-dev'
metric_type = 'custom.googleapis.com/opencensus/gpu_utilization_distribution'

In [38]:
START_TIME = datetime.datetime.fromisoformat('2021-01-22T03:00:00-00:00')
END_TIME = datetime.datetime.fromisoformat('2021-01-22T05:00:00-00:00')

In [42]:
query = Query(client, project_id, metric_type=metric_type)
query = query.select_interval(END_TIME, START_TIME)
#query = query.select_resources(instance_id='1297153725150009955')
#query = query.select_metrics(opencensus_task='py-13989@jk-test')

In [44]:
df = query.as_dataframe()
df

resource_type                                                    gce_instance  \
project_id                                                       jk-mlops-dev   
zone                                                               us-west1-b   
instance_id                                               5382047594078986292   
device                                                                          
opencensus_task                                           py-7103@jk-tf24-gpu   
2021-01-22 03:05:05.414140  count: 12\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:05:35.414293  count: 24\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:06:05.414510  count: 36\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:06:35.414853  count: 48\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:07:05.414989  count: 60\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:07:35.415191  count: 72\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:08:05.415306  count: 84\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:11:01.730146                                                NaN   
2021-01-22 03:11:31.730380                                                NaN   
2021-01-22 03:12:01.730611                                                NaN   
2021-01-22 03:12:31.730768                                                NaN   

resource_type                                                                  \
project_id                                                                      
zone                                                                            
instance_id                                                                     
device                                                                      0   
opencensus_task                                           py-8037@jk-tf24-gpu   
2021-01-22 03:05:05.414140                                                NaN   
2021-01-22 03:05:35.414293                                                NaN   
2021-01-22 03:06:05.414510                                                NaN   
2021-01-22 03:06:35.414853                                                NaN   
2021-01-22 03:07:05.414989                                                NaN   
2021-01-22 03:07:35.415191                                                NaN   
2021-01-22 03:08:05.415306                                                NaN   
2021-01-22 03:11:01.730146  count: 6\nbucket_options {\n  explicit_buckets...   
2021-01-22 03:11:31.730380  count: 12\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:12:01.730611  count: 18\nbucket_options {\n  explicit_bucket...   
2021-01-22 03:12:31.730768  count: 24\nbucket_options {\n  explicit_bucket...   

resource_type                                                                  
project_id                                                                     
zone                                                                           
instance_id                                                                    
device                                                                      1  
opencensus_task                                           py-8037@jk-tf24-gpu  
2021-01-22 03:05:05.414140                                                NaN  
2021-01-22 03:05:35.414293                                                NaN  
2021-01-22 03:06:05.414510                                                NaN  
2021-01-22 03:06:35.414853                                                NaN  
2021-01-22 03:07:05.414989                                                NaN  
2021-01-22 03:07:35.415191                                                NaN  
2021-01-22 03:08:05.415306                                                NaN  
2021-01-22 03:11:01.730146  count: 6\nbucket_options {\n  explicit_buckets...  
2021-01-22 03:11:31.730380  count: 12\nbucket_options {\n  explicit_bucket...  
2021-01-22 03:12:01.730611  count: 18\nbucke